In [1]:
import geemap
import pandas as pd
import numpy as np
import pdb
from IPython.display import display
import ee
import os

In [2]:
ee.Initialize()

In [77]:
# # Define the region of interest for Georgia and Iowa

# georgia = 'F:/US states/georgia.shp'
# iowa = 'F:/US states/iowa.shp'

# georgia_region = geemap.shp_to_ee(georgia)
# iowa_region = geemap.shp_to_ee(iowa)

# #Shapefiles of BMPS

# contour_bufferstrips = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Contour buffer strips 332.shp')
# grassed_waterway = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Grassed waterway 412.shp')
# pond_dam = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Pond dam 378.shp')
# stripcropping = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Stripcropping 585.shp')
# terrace = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Terrace 600.shp')
# basin = geemap.shp_to_ee('F:/Iowa BMP/Iowa shapefiles/Water and sediment control basin.shp')

# # Define the date range for Georgia imagery (most recent)
# georgia_naip_recent = ee.ImageCollection('USDA/NAIP/DOQQ') \
#     .filterBounds(georgia_region) \
#     .filterDate('2017-01-01', '2018-12-31')

# # Define the date range for Iowa imagery (2010)
# iowa_naip_2010 = ee.ImageCollection('USDA/NAIP/DOQQ') \
#     .filterBounds(iowa_region) \
#     .filterDate('2010-01-01', '2010-12-31') \

# # Select bands for true color visualization
# georgia_false_color = georgia_naip_recent.select(['N', 'R', 'G'])
# iowa_true_color = iowa_naip_2010.select(['R', 'G', 'B'])

# # Visualization parameters
# trueColorVis = {
#     'min': 0,
#     'max': 255,
# }

# # Create a map
# Map = geemap.Map(center=[40, -100], zoom=4)

# ## Add layers to the map
# # Map.addLayer(georgia_true_color, trueColorVis, 'Georgia True Color (Recent)')
# # Map.addLayer(iowa_true_color, trueColorVis, 'Iowa True Color (2010)')
# Map.addLayer(stripcropping)

# # Display the map
# # Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# # Define the date range for Georgia imagery (most recent)
# georgia_naip_recent = ee.ImageCollection('USDA/NAIP/DOQQ') \
#     .filterBounds(georgia_region) \
#     .filterDate('2021-01-01', '2022-12-31')

In [ ]:
# georgia_naip_recent

In [ ]:
# iowa_naip_2010

In [ ]:
# Map.addLayer(basin)

# Display the map
# Map

In [78]:
# CLIPPED AOI IN IOWA

# aoi = 'F:/Iowa BMP/Iowa clipped shapefiles/aoi/aoi.shp'
aoi = 'F:/Iowa BMP/Iowa clipped shapefiles/aoi/aoi2.shp' #smaller even than aoi.shp

iowa_aoi = geemap.shp_to_ee(aoi)

#Shapefiles of BMPS

contour_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/contourbuffer_clip.shp')
grassed_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/grassedwaterway_clip.shp')
dam_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/dam_clip.shp')
strip_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/stripcropping_clip.shp')
terrace_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/terrace_clip.shp')
basin_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/bmps_aoi2/basin_clip.shp')

# Define the date range for Georgia imagery (most recent)
# georgia_naip_recent = ee.ImageCollection('USDA/NAIP/DOQQ') \
#     .filterBounds(georgia_region) \
#     .filterDate('2017-01-01', '2018-12-31')

# Define the date range for Iowa imagery (2010)
aoi_naip_2010 = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(iowa_aoi) \
    .filterDate('2010-01-01', '2010-12-31') \

# Select bands for true color visualization
# georgia_false_color = georgia_naip_recent.select(['N', 'R', 'G'])
iowa_false_color = aoi_naip_2010.select(['N', 'R', 'G'])

# Visualization parameters
trueColorVis = {
    'min': 0,
    'max': 255,
}

# Create a map
Map = geemap.Map(center=[40.6, -94], zoom=12)

## Add layers to the map
# Map.addLayer(georgia_true_color, trueColorVis, 'Georgia True Color (Recent)')
Map.addLayer(iowa_false_color, trueColorVis, 'Iowa False Color (2010)')
Map.addLayer(contour_clip, {}, 'contour')
Map.addLayer(grassed_clip, {}, 'grassed waterway')
Map.addLayer(dam_clip, {}, 'dam')
# Map.addLayer(strip_clip, {}, stripcropping) #no stripcropping in aoi2
Map.addLayer(terrace_clip, {}, 'terrace')
Map.addLayer(basin_clip, {}, 'basin')

# Display the map
Map

Map(center=[40.6, -94], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [ ]:
##Articles
#https://www.ingentaconnect.com/contentone/asprs/pers/2017/00000083/00000011/art00011?crawler=true&mimetype=application/pdf
#https://github.com/UGA-Geography-Gengchen-Mai/lab-5-asajulien/blob/main/Lab5_Asa_Julien.ipynb

In [5]:
# iowa_false_color

In [8]:
iowa_list = iowa_false_color.toList(iowa_false_color.size())
iowa_list.size().getInfo()

4

In [54]:
iowa_list

In [59]:
#Add date to image
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

In [75]:
#Extract values from features

def dam_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = dam_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

def terrace_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = terrace_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

def basin_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = basin_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

def strip_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = strip_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

def grassed_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = grassed_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

def contour_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = contour_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

In [63]:
dam_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(dam_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        dam_list.append(sample) 
    except Exception:
        continue
dam_df = pd.concat(dam_list)

    
terrace_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(terrace_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        terrace_list.append(sample) 
    except Exception:
        continue
terrace_df = pd.concat(terrace_list)

basin_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(basin_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        basin_list.append(sample) 
    except Exception:
        continue  
basin_df = pd.concat(basin_list)
        
grassed_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(grassed_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        grassed_list.append(sample)
    except Exception:
        continue
# grassed_df = pd.concat(grassed_list) 

    
contour_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(contour_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        contour_list.append(sample) 
    except Exception:
        continue
# contour_df = pd.concat(contour_list)


#NO STRIPCROPPING IN AOI2
strip_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(strip_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        strip_list.append(sample) 
    except Exception:
        continue
# strip_df = pd.concat(strip_list)

In [84]:
# naip_list = pd.concat([dam_list, terrace_list, basin_list, strip_list, grassed_list, contour_list]) 
##troubles with strip, contour, grassed lists - strip has no data in aoi2, but contour and grassed should

dam_df #need to get rid of repeats - see mosaic-ing section below

,HUC_12,NRCS_CODE,PRACTICE,Present201,CREATOR_NA,SHAPE_Leng,DATE_CREAT,R,imagedate,G,N
0,102801010901,378,Pond Dam,1,CJA,85.268247,2018-05-14,131,20100603,133,174
1,102801010901,378,Pond Dam,1,CJA,85.268247,2018-05-14,141,20100603,144,176
2,102801010901,378,Pond Dam,1,CJA,85.268247,2018-05-14,150,20100603,152,179
3,102801010901,378,Pond Dam,1,CJA,85.268247,2018-05-14,150,20100603,155,180
4,102801010901,378,Pond Dam,1,CJA,85.268247,2018-05-14,149,20100603,156,181
...,...,...,...,...,...,...,...,...,...,...,...
1311,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,156,20100907,143,139
1312,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,133,20100907,136,133
1313,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,96,20100907,129,166
1314,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,119,20100907,134,158


In [74]:
def contour_rasterExtraction(image):
    feature = image.sampleRegions(
        collection = contour_clip, ##BMP HERE
        scale = 1, ##NAIP imagery - check resolution
        geometries = True
    )
    return feature

contour_list = []
for i in range(iowa_list.size().getInfo()):
    try:
        sample = geemap.ee_to_pandas(contour_rasterExtraction(addDate(ee.Image(iowa_list.get(i)))))
        contour_list.append(sample) 
    except Exception:
        continue
contour_df = pd.concat(contour_list)

'''
looking at mosaicing results, this error is due to the sampleRegions function accumulating over 5000 elements for the 
contour and grassed waterway shapefiles. This is probably because they are polygons. Could make a limit to the
number of pixels extracted? See chat gpt answer. Stripcropping didn't have any features in the small aoi2'''

ValueError: No objects to concatenate

In [ ]:
'''
Things to consider:
- Dummy labels to contrast with labeled BMPs. This may help model differentiate BMPs from other farmland
- NLCD farmland and CropScape
- Object based model


With image extraction:
- Add image ID band and mosaic all NAIP imagery with .mosaic function - takes most recent pixel for each point, I think
- With current method: overlap and re-sampling of points. Also how line vs. polygon features are sampled

'''

In [100]:
#Mosaic - NEED TO ADD DATE BAND
aoi_naip_mosaic = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(iowa_aoi) \
    .filterDate('2010-01-01', '2010-12-31') \
    .map(addDate) \
    .mosaic()

dam_mosaic = geemap.ee_to_pandas(dam_rasterExtraction(aoi_naip_mosaic))

dam_mosaic

# contour_mosaic = geemap.ee_to_pandas(contour_rasterExtraction(aoi_naip_mosaic))
# contour_mosaic ##over 5000 elements

,HUC_12,NRCS_CODE,PRACTICE,Present201,CREATOR_NA,SHAPE_Leng,DATE_CREAT,R,B,imagedate,G,N
0,102801010901,378,Pond Dam,1,CJA,43.352763,2018-05-14,121,116,20100907,123,130
1,102801010901,378,Pond Dam,1,CJA,43.352763,2018-05-14,114,113,20100907,116,120
2,102801010901,378,Pond Dam,1,CJA,43.352763,2018-05-14,112,114,20100907,123,127
3,102801010901,378,Pond Dam,1,CJA,43.352763,2018-05-14,123,124,20100907,136,151
4,102801010901,378,Pond Dam,1,CJA,43.352763,2018-05-14,147,125,20100907,139,158
...,...,...,...,...,...,...,...,...,...,...,...,...
1462,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,175,140,20100907,155,150
1463,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,175,143,20100907,152,138
1464,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,183,152,20100907,161,142
1465,102801010901,378,Pond Dam,1,CJA,73.128594,2018-05-14,157,148,20100907,159,157


In [ ]:
# Get the labeled training data for each band
red_train = red[land_label_data>0]
blue_train = blue[land_label_data>0]
green_train = green[land_label_data>0]

xargs = np.column_stack((red_train, blue_train,green_train)) 
# put the three features as three columns of the matrix

# Get the labeled value
yargs = land_label_data[land_label_data>0]

seed =3

# Split to training and test data
from sklearn.model_selection import train_test_split
xargs_train, xargs_test, yargs_train, yargs_test = train_test_split(xargs, yargs, test_size=0.2, random_state=seed)